### Purpose of this notebook is to create series of PSD features for Max Freq and Max Freq value

Will begin with non series and then move to series version(s) of the features

In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


### Load one dataframe

In [2]:
#Load the data and focus on one df specifically

t1 = time.time()

paths , class_list, sleep_night_list , sleep_type_list , participant_id_list = constants.generate_paths_and_info(blue_pebble = True)

path = paths[0]

with io.capture_output() as captured:
    data_epo = mne.read_epochs(path)
    sampling_freq =  data_epo.info['sfreq']  # find sampling frequency
    channels = data_epo.ch_names  # extract channel names
    data = data_epo._data * 1e6  # convert signal from V to uV
    
# Generate dataframe with channels as columns, samples ordered by time _____________________________________________________
channels = constants.channel_list
df_full = data_epo.to_data_frame()
df = df_full[channels].copy()

df

t2 = time.time()

t2-t1

0.3066251277923584

### Gen PSD Features

In [3]:
region_to_channel_dict = constants.region_to_channel_dict

In [9]:
freqs

array([  0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,
         4.5,   5. ,   5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,
         9. ,   9.5,  10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,
        13.5,  14. ,  14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,
        18. ,  18.5,  19. ,  19.5,  20. ,  20.5,  21. ,  21.5,  22. ,
        22.5,  23. ,  23.5,  24. ,  24.5,  25. ,  25.5,  26. ,  26.5,
        27. ,  27.5,  28. ,  28.5,  29. ,  29.5,  30. ,  30.5,  31. ,
        31.5,  32. ,  32.5,  33. ,  33.5,  34. ,  34.5,  35. ,  35.5,
        36. ,  36.5,  37. ,  37.5,  38. ,  38.5,  39. ,  39.5,  40. ,
        40.5,  41. ,  41.5,  42. ,  42.5,  43. ,  43.5,  44. ,  44.5,
        45. ,  45.5,  46. ,  46.5,  47. ,  47.5,  48. ,  48.5,  49. ,
        49.5,  50. ,  50.5,  51. ,  51.5,  52. ,  52.5,  53. ,  53.5,
        54. ,  54.5,  55. ,  55.5,  56. ,  56.5,  57. ,  57.5,  58. ,
        58.5,  59. ,  59.5,  60. ,  60.5,  61. ,  61.5,  62. ,  62.5,
        63. ,  63.5,

In [10]:
win_s = 4
samp_freq = 256
channels = constants.channel_list

win = int(win_s * samp_freq) #Window for Welchs Periodagram
freqs, psd = welch(df.T.values , fs = samp_freq, nperseg=win, average='median')

row_sums = np.sum(psd, axis=1, keepdims=True)
psd_normalised = np.divide(psd, row_sums)

channel_psd_normalised = pd.DataFrame(psd_normalised, index=channels , columns=freqs) #convert psd array into dataframe
region_psd_normalised = eeg_stat_ts.convert_channel_df_to_regions(channel_psd_normalised, region_to_channel_dict)

In [5]:
df_region_psd_normalised = region_psd_normalised
df_region_psd_normalised = channel_psd_normalised

In [6]:
df_region_psd_normalised

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,125.75,126.00,126.25,126.50,126.75,127.00,127.25,127.50,127.75,128.00
Fp1,0.021539,0.186562,0.120327,0.065342,0.044665,0.034631,0.025668,0.015670,0.017788,0.016297,...,2.241958e-13,1.235674e-13,5.957461e-14,1.563725e-14,2.376867e-14,1.149287e-13,1.726978e-13,2.472953e-13,3.320250e-13,1.137550e-13
Fpz,0.014502,0.180344,0.134856,0.062876,0.044398,0.035701,0.027567,0.016050,0.015359,0.012349,...,1.482295e-13,9.628670e-14,3.114156e-14,9.777901e-15,1.973605e-14,8.227630e-14,1.494101e-13,2.052885e-13,3.858024e-13,2.342705e-13
Fp2,0.033295,0.197184,0.118665,0.065659,0.047092,0.029997,0.027120,0.017556,0.016287,0.012046,...,1.645779e-13,1.151231e-13,3.462064e-14,1.088836e-14,2.525471e-14,1.326595e-13,1.895755e-13,2.806634e-13,4.279105e-13,2.495738e-13
F1,0.013783,0.097948,0.057148,0.036126,0.024600,0.020040,0.019683,0.012689,0.012825,0.011684,...,8.706386e-14,4.410338e-14,1.384030e-14,4.495979e-15,1.069865e-14,2.982344e-14,6.736460e-14,9.464981e-14,2.834162e-13,3.468499e-13
Fz,0.020162,0.132771,0.074085,0.039221,0.026697,0.022356,0.015637,0.012771,0.009037,0.011429,...,7.105781e-14,4.842512e-14,1.488533e-14,3.549352e-15,9.182382e-15,3.338467e-14,6.815347e-14,1.193582e-13,1.746093e-13,1.007430e-13
F2,0.016370,0.131322,0.081543,0.038416,0.031211,0.028639,0.017906,0.011814,0.009686,0.010247,...,6.706908e-14,4.473310e-14,2.034369e-14,4.936407e-15,1.207336e-14,3.745213e-14,7.355843e-14,1.009028e-13,4.848843e-13,6.775833e-13
AF7,0.016875,0.128003,0.087509,0.071910,0.053801,0.039031,0.028103,0.020364,0.016155,0.018065,...,1.649393e-13,1.031674e-13,3.345380e-14,9.003782e-15,2.589111e-14,7.270580e-14,1.502383e-13,1.914563e-13,3.766132e-13,2.482109e-13
F7,0.014574,0.137115,0.088024,0.062543,0.045249,0.033508,0.028604,0.022147,0.016907,0.019211,...,2.768018e-13,2.088760e-13,5.992552e-14,1.839843e-14,2.925700e-14,1.298818e-13,2.372878e-13,4.053304e-13,9.761902e-13,5.071917e-13
F5,0.015174,0.104361,0.065568,0.048630,0.039255,0.036821,0.024006,0.015707,0.014704,0.014348,...,2.297374e-13,1.100680e-13,4.137062e-14,1.289549e-14,3.065269e-14,8.002923e-14,1.432453e-13,2.197518e-13,7.257175e-13,4.611610e-13
F3,0.019599,0.114873,0.068768,0.032245,0.032481,0.033955,0.019922,0.014997,0.011875,0.010842,...,9.232019e-14,4.390512e-14,2.301395e-14,5.722348e-15,1.142468e-14,4.086051e-14,9.123538e-14,1.155394e-13,1.701556e-13,9.930116e-14


In [15]:
# Create a new DataFrame with maximum, second-highest, third-highest, and fourth-highest values
# along with their corresponding column names
sorted_values = df_region_psd_normalised.apply(lambda row: sorted(zip(row, row.index), reverse=True), axis=1)
max_values = sorted_values.apply(lambda row: row[0][0])
max_columns = sorted_values.apply(lambda row: row[0][1])
second_max_values = sorted_values.apply(lambda row: row[1][0])
second_max_columns = sorted_values.apply(lambda row: row[1][1])
third_max_values = sorted_values.apply(lambda row: row[2][0])
third_max_columns = sorted_values.apply(lambda row: row[2][1])
fourth_max_values = sorted_values.apply(lambda row: row[3][0])
fourth_max_columns = sorted_values.apply(lambda row: row[3][1])

new_df = pd.DataFrame({'Max_Value': max_values, 'Max_Freq': max_columns,
                        'Second_Max_Value': second_max_values, 'Second_Max_Freq': second_max_columns,
                        'Third_Max_Value': third_max_values, 'Third_Max_Freq': third_max_columns,
                        'Fourth_Max_Value': fourth_max_values, 'Fourth_Max_Freq': fourth_max_columns},
                       index=df_region_psd_normalised.index)
new_df

,Max_Value,Max_Freq,Second_Max_Value,Second_Max_Freq,Third_Max_Value,Third_Max_Freq,Fourth_Max_Value,Fourth_Max_Freq
Fp1,0.186562,0.25,0.120327,0.5,0.065342,0.75,0.044665,1.00
Fpz,0.180344,0.25,0.134856,0.5,0.062876,0.75,0.044398,1.00
Fp2,0.197184,0.25,0.118665,0.5,0.065659,0.75,0.047092,1.00
F1,0.097948,0.25,0.057148,0.5,0.036126,0.75,0.032692,7.00
Fz,0.132771,0.25,0.074085,0.5,0.039221,0.75,0.029109,7.00
F2,0.131322,0.25,0.081543,0.5,0.038416,0.75,0.031211,1.00
AF7,0.128003,0.25,0.087509,0.5,0.071910,0.75,0.053801,1.00
F7,0.137115,0.25,0.088024,0.5,0.062543,0.75,0.045249,1.00
F5,0.104361,0.25,0.065568,0.5,0.048630,0.75,0.039255,1.00
F3,0.114873,0.25,0.068768,0.5,0.033955,1.25,0.032481,1.00


In [13]:
new_df.columns

Index(['Max_Value', 'Max_Freq', 'Second_Max_Value', 'Second_Max_Freq',
       'Third_Max_Value', 'Third_Max_Freq', 'Fourth_Max_Value',
       'Fourth_Max_Freq'],
      dtype='object')

In [9]:
new_df['window_no.'] = 1

In [ ]:
#4. Convert into a single row of a new dataframe where each cell is a series
new_row = eeg_stat_ts.dataframe_to_ts_row(new_df, list(new_df.columns)[:-1] )

In [16]:
def gen_max_psd_properties(df, channels, win_s = 4, samp_freq = 256):
    '''
    At the moment this defaults to calculating the top 4 Max freqs and corresponding Max Freq values 
    Will make this variable in the future
    
    Inputs: 
    - df, dataframe with EEG channels as columns 
    - channels, list -> EEG channel names 
    - win_s --> Window size in seconds for calculation of Welchs Periodogram
    - samp_freq --> Sampling frequency in Hz
    
    Output:
    - Dataframe - with columns as Max value(s) in normalised PSD and corresponding Max Frequencies calculated from input df per channel... 
    ...cols are ->['Max_Value', 'Max_Freq', 'Second_Max_Value', 'Second_Max_Freq','Third_Max_Value', 'Third_Max_Freq', 'Fourth_Max_Value',
                   'Fourth_Max_Freq']...
   
      Row Index is the channels (Fp1,Fpz etc...)    
    '''

    win = int(win_s * samp_freq) #Window for Welchs Periodagram
    freqs, psd = welch(df.T.values , fs = samp_freq, nperseg=win, average='median')

    row_sums = np.sum(psd, axis=1, keepdims=True)
    psd_normalised = np.divide(psd, row_sums)

    channel_psd_normalised = pd.DataFrame(psd_normalised, index=channels , columns=freqs) #convert psd array into dataframe

    #From the normalised PSD Dataframe, caluclate the max power values and corresponding frequency values
    # Create a new DataFrame with maximum, second-highest, third-highest, and fourth-highest values
    # along with their corresponding column names
    sorted_values = channel_psd_normalised.apply(lambda row: sorted(zip(row, row.index), reverse=True), axis=1)
    
    max_values = sorted_values.apply(lambda row: row[0][0])
    max_columns = sorted_values.apply(lambda row: row[0][1])
    
    second_max_values = sorted_values.apply(lambda row: row[1][0])
    second_max_columns = sorted_values.apply(lambda row: row[1][1])
    
    third_max_values = sorted_values.apply(lambda row: row[2][0])
    third_max_columns = sorted_values.apply(lambda row: row[2][1])
    
    fourth_max_values = sorted_values.apply(lambda row: row[3][0])
    fourth_max_columns = sorted_values.apply(lambda row: row[3][1])
    
    new_df = pd.DataFrame({'Max_Value': max_values, 'Max_Freq': max_columns,
                            'Second_Max_Value': second_max_values, 'Second_Max_Freq': second_max_columns,
                            'Third_Max_Value': third_max_values, 'Third_Max_Freq': third_max_columns,
                            'Fourth_Max_Value': fourth_max_values, 'Fourth_Max_Freq': fourth_max_columns},
                           index=channel_psd_normalised.index)

    return new_df